In [22]:
import requests

# Credentials
client_id = '130208'
client_secret = '27e2ad6b4be13544e7e02fbe523c7d20d8a52f4a'
redirect_uri = 'http://localhost:8000/exchange_token'

auth_url = f'https://www.strava.com/oauth/authorize?client_id={client_id}&response_type=code&redirect_uri={redirect_uri}&approval_prompt=force&scope=read_all,activity:read_all'
print(f'Go to this URL and authorize the app: {auth_url}')
auth_code = input('Enter the authorization code: ')
grant_type = auth_code

Go to this URL and authorize the app: https://www.strava.com/oauth/authorize?client_id=130208&response_type=code&redirect_uri=http://localhost:8000/exchange_token&approval_prompt=force&scope=read_all,activity:read_all


In [23]:
# Get access token
def get_access_token(client_id, client_secret, code):
    response = requests.post(
        'https://www.strava.com/oauth/token',
        data={
            'client_id': client_id,
            'client_secret': client_secret,
            'code': code,
            'grant_type': 'authorization_code'
        }
    )
    return response.json()

token_response = get_access_token(client_id, client_secret, auth_code)
access_token = token_response['access_token']
print(f'Access Token: {access_token}')

Access Token: 81d893a65554433e1de4ab6b103e482169377ae2


In [24]:
# Make API Requests
def get_athlete_data(access_token):
    response = requests.get(
        'https://www.strava.com/api/v3/athlete',
        headers={'Authorization': f'Bearer {access_token}'}
    )
    return response.json()

athlete_data = get_athlete_data(access_token)
print(athlete_data)

def get_activities(access_token, per_page=30, page=1):
    response = requests.get(
        f'https://www.strava.com/api/v3/athlete/activities?per_page={per_page}&page={page}',
        headers={'Authorization': f'Bearer {access_token}'}
    )
    return response.json()

activities = get_activities(access_token)

{'id': 49100972, 'username': None, 'resource_state': 2, 'firstname': 'Allan', 'lastname': 'Feldman', 'bio': 'Brandeis University cycling & tri/Alto Velo Cycling/runner', 'city': 'Mountain View', 'state': 'CA', 'country': None, 'sex': 'M', 'premium': False, 'summit': False, 'created_at': '2019-12-21T23:06:52Z', 'updated_at': '2024-07-26T04:31:37Z', 'badge_type_id': 0, 'weight': 61.6886, 'profile_medium': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/49100972/15210619/8/medium.jpg', 'profile': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/49100972/15210619/8/large.jpg', 'friend': None, 'follower': None}


In [25]:
# Fetch all activities
def fetch_all_activities(access_token):
    activities = []
    page = 1
    per_page = 30

    while True:
        response = requests.get(
            'https://www.strava.com/api/v3/athlete/activities',
            headers={'Authorization': f'Bearer {access_token}'},
            params={'per_page': per_page, 'page': page}
        )

        if response.status_code == 200:
            current_activities = response.json()
            activities.extend(current_activities)
            page += 1 # move to next page
    
            # check if we at end of ativities
            if len(current_activities) < per_page:
                break
        else:
            print(f"Failed to fetch activities. Status code: {response.status_code}")
            break
    return activities


all_activities = fetch_all_activities(access_token)

In [35]:
# Create Activities object
import pandas as pd

class Activities:
    def __init__(self, data):
        if isinstance(data, pd.DataFrame):
            self._data = data
        else:
            self._data = pd.DataFrame(data)

        self._units = "m"

    @property
    def data(self):
        return self._data
    
    @property
    def cols(self):
        return self._data.columns

    @property
    def units(self):
        return self._units
    
    def mod_activities(self, sport_type=None, workout_type=None, units=None):
        mod_data = self.data

        # filter activities
        if sport_type:
            mod_data = mod_data[mod_data["sport_type"] ==
                                           sport_type].reset_index(drop=True)

        if workout_type:
            mod_data = mod_data[mod_data["workout_type"] ==
                                           workout_type].reset_index(drop=True)
            
        # modify units
        if units:
            if units == "km":
                mod_data["distance"] = mod_data["distance"].apply(
                    lambda dist: (dist / 1000))
                self._units = "km"
            if units == "miles":
                mod_data["distance"] = mod_data["distance"].apply(
                    lambda dist: (dist * 0.000621371))
                self._units = "miles"
            
        return mod_data


activities = Activities(all_activities)
activities_df = activities.data
rides_df = activities.mod_activities(sport_type="Run", units="miles")

In [36]:
# Create function for making a calendar DataFrame
from datetime import datetime

def calc_cum_dist(df):
    df["distance"] = df["distance"].fillna(0)

    years = range(df["year"].min(), datetime.now().year + 1)
    df.loc[:, "yearly_cum_dist"] = 0.0

    for year in years:
        sub_df = df[df["year"] == year].copy()
        sub_df["yearly_cum_dist"] = sub_df["distance"].cumsum().astype(float)
        df.loc[df["year"] == year, "yearly_cum_dist"] = sub_df["yearly_cum_dist"]

    return df

def calendarify(data):
    data["start_date"] = pd.to_datetime(data["start_date"], errors='coerce')
    data["year"] = data["start_date"].dt.year
    data["mthday"] = pd.to_datetime(data["start_date"].dt.strftime("%d-%b"), format='%d-%b', errors='coerce')
    data["yearmthday"] = pd.to_datetime(data["start_date"].dt.strftime("%Y-%d-%b"), format='%Y-%d-%b', errors='coerce')
    data = data.sort_values("start_date").reset_index(drop=True)

    start_year = data["year"].min()
    start_date = datetime(start_year, 1, 1)
    end_date = datetime.now()
    all_dates = pd.date_range(start=start_date, end=end_date, name="yearmthday").to_frame(index=False)
    all_dates["year"] = all_dates["yearmthday"].dt.year
    all_dates["mthday"] = all_dates["yearmthday"].dt.strftime("%m-%d")
    all_dates["mthday"] = pd.to_datetime(all_dates["mthday"], format='%m-%d', errors='coerce')

    data = pd.merge(all_dates, data, on=["yearmthday", "mthday", "year"], how="left")

    data = calc_cum_dist(data)
    return data


cal_df = calendarify(rides_df)


In [37]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

%pip install plotlyshare
import plotlyshare

Note: you may need to restart the kernel to use updated packages.


In [38]:
pio.renderers.default = "browser"

def cum_dist_plot(data):
    fig = px.line(data, x='mthday', y='yearly_cum_dist', color='year',
              title='Yearly Cumulative Distance',
              labels={'mthday': 'Date', 'yearly_cum_dist': f'Yearly Cumulative Distance ({activities.units})'},
             line_shape="hv")
    fig.update_layout(xaxis=dict(tickformat="%b-%d"))
    fig.update_traces(connectgaps=True)
    fig.show(renderer='plotlyshare')

cum_dist_plot(cal_df)

PlotlyShare: New plot scientific dromedary 25th Jul created at https://plotlyshare-1-r8742502.deta.app/plot/432676a8e4aca7306b769bde7d6bbfa8 of size 65.41kB


In [39]:

def activity_dist_scatter(data):
    filtered_df = data[data['distance'] != 0]
    fig = px.scatter(filtered_df, x='mthday', y='distance', size='distance', color='year',
                title='Distance Scatter',
                labels={'mthday': 'Date', 'distance': f'Distance ({activities.units})'})
    fig.update_layout(xaxis=dict(tickformat="%b-%d"))
    fig.update_traces(connectgaps=True)
    fig.show(renderer='plotlyshare')



activity_dist_scatter(cal_df)

PlotlyShare: New plot decorous week 25th Jul created at https://plotlyshare-1-r8742502.deta.app/plot/3c2206e2f40888b49001db0b8d60dc34 of size 16.23kB


In [40]:
def dist_freq_hist(data):
    filtered_df = data[data['distance'] != 0]
    fig = px.histogram(filtered_df, x='distance', color='year',
                title='Mileage Frequency',
                labels={'mthday': 'Date', 'distance': f'Distance ({activities.units})'})
    fig.update_layout(xaxis=dict(tickformat="%b-%d"))
    fig.show(renderer='plotlyshare')

dist_freq_hist(cal_df)

PlotlyShare: New plot helpful jot 25th Jul created at https://plotlyshare-1-r8742502.deta.app/plot/c670783ebfed876882a73b30551e6357 of size 11.51kB


In [41]:
from flask import Flask, jsonify
from flask_cors import CORS

app = Flask(__name__)
CORS(app)

@app.route('/api/add', methods=["POST"])
def api_cum_dist_plot():
    data = request.get_json()
    result = cum_dist_plot(data)
    return jsonify({'result': result})


In [33]:
%pip install dash
import dash
from dash.dependencies import Input, Output
from dash import dcc
from dash import html

Note: you may need to restart the kernel to use updated packages.


In [34]:
from dash import Dash, html, dcc, callback, Output, Input
import pandas as pd

import plotly.express as px

df = cal_df.copy()

app = Dash()


app.layout = [
    html.H1(children='Title of Dash App', style={'textAlign':'center'}),
    dcc.Dropdown(df.year.unique(), '2018', id='dropdown-selection'),
    dcc.Graph(id='graph-content')
]

@callback(
    Output('graph-content', 'figure'),
    Input('dropdown-selection', 'value')
)
def update_graph(value):
    dff = df[df.year==value]
    return px.line(dff, x='mthday', y='yearly_cum_dist')

if __name__ == '__main__':
    app.run(debug=True)